In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '7'

In [3]:
from deeprte.config import get_config
config = get_config()

In [4]:
source_dir = "/workspaces/deeprte/rte_data/rte_data/matlab/train-delta"
data_name_list = ["train_delta_1.mat","train_delta_2.mat"]

In [4]:
from deeprte.data.pipeline import DataPipeline
data_pipeline = DataPipeline(source_dir, data_name_list)
data = data_pipeline.process(pre_shuffle=True, is_split_test_samples=True, num_test_samples= 2)

from deeprte.model.tf.rte_dataset import np_to_tensor_dict

tf_data = np_to_tensor_dict(data)

from deeprte.model.tf.rte_dataset import divide_batch_feat
batched_data, unbatched_data = divide_batch_feat(tf_data)

In [5]:
import tensorflow as tf

tf.nest.map_structure(lambda x:x.shape, tf_data)

{'sigma': TensorShape([998, 1600, 2]),
 'psi_label': TensorShape([998, 38400]),
 'scattering_kernel': TensorShape([998, 38400, 24]),
 'self_scattering_kernel': TensorShape([998, 24, 24]),
 'boundary': TensorShape([998, 1920]),
 'position_coords': TensorShape([1600, 2]),
 'velocity_coords': TensorShape([24, 2]),
 'phase_coords': TensorShape([38400, 4]),
 'boundary_coords': TensorShape([1920, 4]),
 'boundary_weights': TensorShape([1920]),
 'velocity_weights': TensorShape([24])}

In [6]:
from deeprte.model.tf.input_pipeline import split_feat,process_features, make_device_batch
import jax
train_feat = split_feat(tf_data, 0.8, True)
# ds, unbatched_feat = load_and_split_data(tf_data, data_config, True)

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
tf.nest.map_structure(lambda x:x.shape, train_feat)

{'sigma': TensorShape([6, 1681, 2]),
 'psi_label': TensorShape([6, 40344]),
 'scattering_kernel': TensorShape([6, 40344, 24]),
 'self_scattering_kernel': TensorShape([6, 24, 24]),
 'boundary': TensorShape([6, 1968]),
 'position_coords': TensorShape([1681, 2]),
 'velocity_coords': TensorShape([24, 2]),
 'phase_coords': TensorShape([40344, 4]),
 'boundary_coords': TensorShape([1968, 4]),
 'boundary_weights': TensorShape([1968]),
 'velocity_weights': TensorShape([24])}

In [8]:
data_config = config.dataset

In [9]:
train_ds = process_features(train_feat, is_training=True, batch_sizes= make_device_batch(data_config.train.batch_size, jax.device_count()),collocation_sizes=data_config.train.collocation_sizes)
# train_ds = process_features(train_feat, is_training=True, batch_sizes= [2,3],collocation_sizes=data_config.train.collocation_sizes)

In [10]:
tf.nest.map_structure(lambda x:x.shape, next(train_ds))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


{'sigma': (1, 6, 1681, 2),
 'psi_label': (1, 6, 500),
 'scattering_kernel': (1, 6, 500, 24),
 'self_scattering_kernel': (1, 6, 24, 24),
 'boundary': (1, 6, 1968),
 'position_coords': (1, 1681, 2),
 'velocity_coords': (1, 24, 2),
 'phase_coords': (1, 500, 4),
 'boundary_coords': (1, 1968, 4),
 'boundary_weights': (1, 1968),
 'velocity_weights': (1, 24)}

In [11]:
val_feat = split_feat(tf_data, 0.8, False)

In [12]:
val_ds = process_features(val_feat, is_training=False, batch_sizes= make_device_batch(data_config.validation.batch_size, jax.device_count()))

In [13]:
tf.nest.map_structure(lambda x:x.shape, next(val_ds))

{'sigma': (1, 2, 1681, 2),
 'psi_label': (1, 2, 40344),
 'scattering_kernel': (1, 2, 40344, 24),
 'self_scattering_kernel': (1, 2, 24, 24),
 'boundary': (1, 2, 1968),
 'position_coords': (1, 1681, 2),
 'velocity_coords': (1, 24, 2),
 'phase_coords': (1, 40344, 4),
 'boundary_coords': (1, 1968, 4),
 'boundary_weights': (1, 1968),
 'velocity_weights': (1, 24)}

In [5]:
from deeprte.model.tf.input_pipeline import load_tf_data, tf_data_to_generator

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
ds = load_tf_data(source_dir, data_name_list, pre_shuffle=True,is_split_test_samples=True, num_test_samples = 2)

In [8]:
import tensorflow as tf

In [9]:
tf.nest.map_structure(lambda x:x.shape, ds)

{'sigma': TensorShape([998, 1600, 2]),
 'psi_label': TensorShape([998, 38400]),
 'scattering_kernel': TensorShape([998, 38400, 24]),
 'self_scattering_kernel': TensorShape([998, 24, 24]),
 'boundary': TensorShape([998, 1920]),
 'position_coords': TensorShape([1600, 2]),
 'velocity_coords': TensorShape([24, 2]),
 'phase_coords': TensorShape([38400, 4]),
 'boundary_coords': TensorShape([1920, 4]),
 'boundary_weights': TensorShape([1920]),
 'velocity_weights': TensorShape([24])}

In [18]:
tfds = tf_data_to_generator(ds,True, [2,3],0.8,collocation_sizes=500)

In [19]:
tf.nest.map_structure(lambda x:x.shape, next(tfds))

{'sigma': (2, 3, 1681, 2),
 'psi_label': (2, 3, 500),
 'scattering_kernel': (2, 3, 500, 24),
 'self_scattering_kernel': (2, 3, 24, 24),
 'boundary': (2, 3, 1968),
 'position_coords': (2, 1681, 2),
 'velocity_coords': (2, 24, 2),
 'phase_coords': (2, 500, 4),
 'boundary_coords': (2, 1968, 4),
 'boundary_weights': (2, 1968),
 'velocity_weights': (2, 24)}